In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date
pd.options.display.max_rows = 100


C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [2]:
#read in the data
mid = pd.read_csv("experiment MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)


#time["Elapsed time, mass 40"].describe()

       mass 2.0_value  mass 4.0_value  mass 5.5_value  mass 12.0_value  \
count    4.610000e+02    4.610000e+02    4.610000e+02     4.610000e+02   
mean     2.241549e-11    1.087270e-12    1.839111e-14     1.877213e-13   
std      8.715757e-13    3.059772e-14    5.074247e-15     1.440962e-14   
min      2.161390e-11    1.025200e-12    8.900000e-15     1.201000e-13   
25%      2.201010e-11    1.068300e-12    1.530000e-14     1.783000e-13   
50%      2.222560e-11    1.083900e-12    1.790000e-14     1.870000e-13   
75%      2.276520e-11    1.105700e-12    2.070000e-14     1.971000e-13   
max      3.829790e-11    1.453100e-12    7.310000e-14     2.843000e-13   

       mass 15.0_value  mass 16.0_value  mass 18.0_value  mass 28.0_value  \
count     4.610000e+02     4.610000e+02     4.610000e+02     4.610000e+02   
mean      3.657592e-14     4.559069e-13     8.521360e-13     9.560581e-13   
std       2.406796e-14     1.973723e-14     3.049056e-14     4.995829e-14   
min       1.550000e-14   

,mass 2.0_time,mass 4.0_time,mass 5.5_time,mass 12.0_time,mass 15.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,...,"Elapsed time, mass 4","Elapsed time, mass 5.5","Elapsed time, mass 12","Elapsed time, mass 15","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2022/12/13 16:41:38.247,2022/12/13 16:41:38.497,2022/12/13 16:41:38.747,2022/12/13 16:41:38.997,2022/12/13 16:41:39.247,2022/12/13 16:41:39.497,2022/12/13 16:41:39.748,2022/12/13 16:41:39.998,2022/12/13 16:41:40.248,2022/12/13 16:41:40.499,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN
3,2022/12/13 16:41:44.572,2022/12/13 16:41:44.822,2022/12/13 16:41:45.072,2022/12/13 16:41:45.322,2022/12/13 16:41:45.572,2022/12/13 16:41:45.822,2022/12/13 16:41:46.073,2022/12/13 16:41:46.323,2022/12/13 16:41:46.573,2022/12/13 16:41:46.824,...,6.325,6.325,6.325,6.325,6.325,6.325,6.325,6.325,6.325,NaN
4,2022/12/13 16:41:50.897,2022/12/13 16:41:51.147,2022/12/13 16:41:51.397,2022/12/13 16:41:51.647,2022/12/13 16:41:51.897,2022/12/13 16:41:52.148,2022/12/13 16:41:52.398,2022/12/13 16:41:52.648,2022/12/13 16:41:52.899,2022/12/13 16:41:53.149,...,12.650,12.650,12.650,12.650,12.651,12.650,12.650,12.651,12.650,NaN
5,2022/12/13 16:41:57.222,2022/12/13 16:41:57.472,2022/12/13 16:41:57.722,2022/12/13 16:41:57.972,2022/12/13 16:41:58.222,2022/12/13 16:41:58.473,2022/12/13 16:41:58.723,2022/12/13 16:41:58.973,2022/12/13 16:41:59.224,2022/12/13 16:41:59.474,...,18.975,18.975,18.975,18.975,18.976,18.975,18.975,18.976,18.975,NaN
6,2022/12/13 16:42:03.547,2022/12/13 16:42:03.797,2022/12/13 16:42:04.047,2022/12/13 16:42:04.297,2022/12/13 16:42:04.547,2022/12/13 16:42:04.798,2022/12/13 16:42:05.048,2022/12/13 16:42:05.298,2022/12/13 16:42:05.549,2022/12/13 16:42:05.799,...,25.300,25.300,25.300,25.300,25.301,25.300,25.300,25.301,25.300,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,2022/12/13 17:29:42.447,2022/12/13 17:29:42.697,2022/12/13 17:29:42.947,2022/12/13 17:29:43.197,2022/12/13 17:29:43.447,2022/12/13 17:29:43.698,2022/12/13 17:29:43.948,2022/12/13 17:29:44.198,2022/12/13 17:29:44.449,2022/12/13 17:29:44.699,...,2884.200,2884.200,2884.200,2884.200,2884.201,2884.200,2884.200,2884.201,2884.200,NaN
459,2022/12/13 17:29:48.772,2022/12/13 17:29:49.022,2022/12/13 17:29:49.272,2022/12/13 17:29:49.522,2022/12/13 17:29:49.772,2022/12/13 17:29:50.023,2022/12/13 17:29:50.273,2022/12/13 17:29:50.524,2022/12/13 17:29:50.774,2022/12/13 17:29:51.024,...,2890.525,2890.525,2890.525,2890.525,2890.526,2890.525,2890.526,2890.526,2890.525,NaN
460,2022/12/13 17:29:55.097,2022/12/13 17:29:55.347,2022/12/13 17:29:55.597,2022/12/13 17:29:55.847,2022/12/13 17:29:56.097,2022/12/13 17:29:56.348,2022/12/13 17:29:56.598,2022/12/13 17:29:56.848,2022/12/13 17:29:57.099,2022/12/13 17:29:57.349,...,2896.850,2896.850,2896.850,2896.850,2896.851,2896.850,2896.850,2896.851,2896.850,NaN
461,2022/12/13 17:30:01.422,2022/12/13 17:30:01.672,2022/12/13 17:30:01.922,2022/12/13 17:30:02.172,2022/12/13 17:30:02.422,2022/12/13 17:30:02.673,2022/12/13 17:30:02.923,2022/12/13 17:30:03.173,2022/12/13 17:30:03.424,2022/12/13 17:30:03.674,...,2903.175,2903.175,2903.175,2903.175,2903.176,2903.175,2903.175,2903.176,2903.175,NaN


In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,13/12/2022 16:40:04,7.0,-0.046909,3.760000e-09,1.860000e-09,4.996000e-09,4.919000e-07,0.35235,207.0,-5.387,4280.342,NaN,NaN
2,NaN,13/12/2022 16:40:11,14.0,-0.046614,2.990000e-09,1.580000e-09,4.996000e-09,4.919000e-07,0.35236,207.0,-5.050,4280.342,NaN,NaN
3,NaN,13/12/2022 16:40:18,21.0,-0.046566,3.380000e-09,1.710000e-09,4.996000e-09,4.913000e-07,0.35236,206.0,-5.050,4280.791,NaN,NaN
4,NaN,13/12/2022 16:40:25,28.0,-0.046811,2.290000e-09,8.920000e-10,4.996000e-09,4.913000e-07,0.35236,207.0,-5.387,4280.566,NaN,NaN
5,NaN,13/12/2022 16:40:32,35.0,-0.046417,1.250000e-09,5.480000e-10,4.996000e-09,4.919000e-07,0.35234,206.0,-5.163,4280.679,NaN,NaN


In [4]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])
hv.insert(0,"datetime",hv["Date"]+" "+hv["Time"])
print(hv.head())
#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.describe()


                  datetime        Date          Time  VMon1   VMon2    VMon3  \
0  13/12/2022 16:38:03.758  13/12/2022  16:38:03.758   0.14  100.08  1000.04   
1  13/12/2022 16:38:04.956  13/12/2022  16:38:04.956   0.14  100.08  1000.04   
2  13/12/2022 16:40:53.207  13/12/2022  16:40:53.207   0.08    0.06     0.08   
3  13/12/2022 16:40:54.393  13/12/2022  16:40:54.393   0.08    0.06     0.08   
4  13/12/2022 16:40:56.610  13/12/2022  16:40:56.610   0.08    0.06     0.08   

     VMon4    VMon5  VMon6  VMon7  ...  IMon2    IMon3    IMon4    IMon5  \
0  1000.18  1000.18    0.0    0.0  ...  1.373  34.3760  34.3981  34.3979   
1  1000.18  1000.18    0.0    0.0  ...  1.377  34.3761  34.4017  34.3984   
2     0.08     0.00    0.0    0.0  ...  0.000   0.1312   0.0526   0.0983   
3     0.08     0.00    0.0    0.0  ...  0.000   0.1312   0.0526   0.0983   
4     0.08     0.00    0.0    0.0  ...  0.000   0.1312   0.0526   0.0983   

    IMon6   IMon7   IMon8  Comment       hv_grid        em  
0

,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,hv_grid,em
count,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.0,2456.0,2.456000e+03,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2456.000000,2.456000e+03,2.456000e+03
mean,1643.923447,165.992573,81.647484,814.144560,813.629536,725.277915,0.0,0.0,2.000000e-02,-0.034670,0.257568,26.050754,26.064608,26.052901,-0.012822,-0.028052,-0.011060,5.130187e-08,2.753339e-07
std,852.556416,234.804849,38.763412,387.699055,388.187202,445.265767,0.0,0.0,8.467175e-16,0.071952,0.360388,12.914011,12.923975,12.922912,0.001148,0.002000,0.001669,6.120949e-08,3.469987e-07
min,0.000000,0.040000,0.060000,0.000000,0.000000,0.000000,0.0,0.0,2.000000e-02,-0.666100,-2.880800,-0.515900,-0.569200,-0.367000,-0.015200,-0.030600,-0.013600,0.000000e+00,0.000000e+00
25%,906.133000,0.060000,100.060000,999.980000,1000.040000,0.000000,0.0,0.0,2.000000e-02,-0.099425,0.015300,20.552725,20.568075,20.513600,-0.013000,-0.029900,-0.012400,3.500000e-09,1.612500e-08
50%,1643.663500,0.120000,100.080000,1000.040000,1000.160000,1000.140000,0.0,0.0,2.000000e-02,-0.002900,0.018600,33.550650,33.569250,33.562000,-0.012700,-0.027800,-0.010900,1.970000e-08,1.860000e-08
75%,2381.312250,499.920000,100.100000,1000.040000,1000.160000,1000.160000,0.0,0.0,2.000000e-02,0.017600,0.630750,33.619200,33.637800,33.628100,-0.012000,-0.026700,-0.009700,9.962500e-08,6.359000e-07
max,3118.554000,499.980000,100.180000,1000.120000,1000.220000,1000.480000,0.0,0.0,2.000000e-02,0.538900,2.759900,34.376100,34.401700,34.398400,-0.011200,-0.023900,-0.007400,6.661000e-07,2.880800e-06


# Data processing

In [5]:
#Filtering background signal and aligning data

#Find the timestamp where emission was turned on and off
print("Result 1:",hv.loc[hv["Comment"]=="emission on"])
print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
x=pd.Timestamp(2022, 12, 13, 16,50,28)
y=pd.Timestamp(2022, 12, 13, 16,54, 32)

#then find the time value closest to it in the MID scan file 
print("\n","Result 3:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 5.5_value"])

#apply background correction to the rest of the mass spectrum
mid["mass 2 corr"]=mid["mass 2.0_value"]-bck

#account for the thermal transpiration factor
T2 = 4.2
T1 = 296.5
T_coef = np.sqrt(T2/T1)
mid["mass 2 corr"] = mid["mass 2 corr"]*T_coef

#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2 corr"][86:124]/(h2_sens*0.46)
print(p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2 corr"][0:86]/(h2_sens*0.46)
base_h2.describe()

Result 1:                     datetime        Date  Elapsed time          Time  VMon1  \
338  13/12/2022 16:47:38.667  13/12/2022       574.909  16:47:38.667   0.06   
339  13/12/2022 16:47:39.861  13/12/2022       576.103  16:47:39.861   0.06   
340  13/12/2022 16:47:41.059  13/12/2022       577.301  16:47:41.059   0.06   
341  13/12/2022 16:47:42.251  13/12/2022       578.493  16:47:42.251   0.06   
479  13/12/2022 16:50:28.222  13/12/2022       744.464  16:50:28.222   0.12   
480  13/12/2022 16:50:29.391  13/12/2022       745.633  16:50:29.391   0.10   
481  13/12/2022 16:50:30.584  13/12/2022       746.826  16:50:30.584   0.12   
482  13/12/2022 16:50:31.770  13/12/2022       748.012  16:50:31.770   0.12   

      VMon2   VMon3    VMon4   VMon5  VMon6  ...   IMon2    IMon3    IMon4  \
338  100.10  999.92   999.96    0.00    0.0  ...  0.0118  20.4542  20.4598   
339  100.10  999.92   999.96    0.00    0.0  ...  0.0118  20.4550  20.4645   
340  100.10  999.92   999.96    0.00    0.0 

count    8.600000e+01
mean     2.681651e-10
std      5.140469e-12
min      2.635911e-10
25%      2.662504e-10
50%      2.671891e-10
75%      2.685186e-10
max      3.021717e-10
Name: mass 2 corr, dtype: float64

## Point-by-point desorption

Couple of approaches were investigated to align hv and mid datasets both with different sampling rates, starting times and lengths. 

The larger dataset with the higher sampling rate was reindexed to fit the mid-dataset with fewer points and lower sampling rate.



In [6]:
#make a sub-df from mid that includes only mass 2 data
h2 = mid.loc[:, ["mass 2.0_time", "mass 2.0_value"]]
#rename columns
h2 = h2.rename(columns={"mass 2.0_time": "datetime","mass 2.0_value": "Ion current"}).reset_index(drop=True)

print(h2)

                    datetime   Ion current
0    2022/12/13 16:41:38.247  2.195250e-11
1    2022/12/13 16:41:44.572  2.178220e-11
2    2022/12/13 16:41:50.897  2.171820e-11
3    2022/12/13 16:41:57.222  2.174880e-11
4    2022/12/13 16:42:03.547  2.194720e-11
..                       ...           ...
456  2022/12/13 17:29:42.447  2.179670e-11
457  2022/12/13 17:29:48.772  2.179670e-11
458  2022/12/13 17:29:55.097  2.186220e-11
459  2022/12/13 17:30:01.422  2.167730e-11
460  2022/12/13 17:30:07.747  2.168000e-11

[461 rows x 2 columns]


In [7]:
# Convert the 'time' column to datetime index for both dataframes
hv.index = (pd.to_datetime(hv['datetime']).dt.round(freq="S"))
h2.index = (pd.to_datetime(h2['datetime']).dt.round(freq="S"))
print(hv)

# Reindex df1 to match the datetime index of df2
hv = hv.reindex(h2.index, method='nearest', limit=1)

#Drop the "datetime" columns
hv.drop('datetime', axis=1, inplace=True)
h2.drop('datetime', axis=1, inplace=True)

# Now df1 and df2 have the same datetime index and can be joined or aligned
# by joining on the index
hv_aligned = hv.merge(h2,left_index=True, right_index=True, how="inner")

print("aligned",hv_aligned)

                                    datetime        Date  Elapsed time  \
datetime                                                                 
2022-12-13 16:38:04  13/12/2022 16:38:03.758  13/12/2022         0.000   
2022-12-13 16:38:05  13/12/2022 16:38:04.956  13/12/2022         1.198   
2022-12-13 16:40:53  13/12/2022 16:40:53.207  13/12/2022       169.449   
2022-12-13 16:40:54  13/12/2022 16:40:54.393  13/12/2022       170.635   
2022-12-13 16:40:57  13/12/2022 16:40:56.610  13/12/2022       172.852   
...                                      ...         ...           ...   
2022-12-13 17:29:58  13/12/2022 17:29:57.571  13/12/2022      3113.813   
2022-12-13 17:29:59  13/12/2022 17:29:58.761  13/12/2022      3115.003   
2022-12-13 17:30:00  13/12/2022 17:29:59.938  13/12/2022      3116.180   
2022-12-13 17:30:01  13/12/2022 17:30:01.128  13/12/2022      3117.370   
2022-12-13 17:30:02  13/12/2022 17:30:02.312  13/12/2022      3118.554   

                             Time  VM

In [8]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [9]:
#Filtering background signal for interpolated datasets

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 5.5_value"])

#apply background correction to the rest of the mass spectrum
hv_aligned["mass 2 corr"]=hv_aligned["Ion current"]-bck

#From the signal, get the partial pressures 
h2_sens=2.13e-2 #A/mbar
p_h2_ip=hv_aligned["mass 2 corr"]/(h2_sens*0.46) #H2 equivalent pressures

#account for the thermal transpiration factor
T2 = 4.2
T1 = 296.5
T_coef = np.sqrt(T2/T1)
p_h2_ip = p_h2_ip*T_coef 

#insert partial pressures into the dataframe
hv_aligned.insert(22,"hydrogen pp",p_h2_ip)


In [10]:
hv_aligned.fillna(0,inplace=True)
hv_aligned["hydrogen pp"].describe()

count    4.610000e+02
mean     2.720612e-10
std      1.058717e-11
min      2.623242e-10
25%      2.671369e-10
50%      2.697546e-10
75%      2.763092e-10
max      4.649874e-10
Name: hydrogen pp, dtype: float64

### Plotting

In [11]:
#plot the aligned dataframe results
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.plot(hv_aligned["Elapsed time"],hv_aligned["em"],linestyle="-",marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA in 80mm, dynamic vacuum")


plt.yscale('log')


In [12]:
#plot the h2 mid 
#seems to be too little rows?
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA in 80mm, dynamic vacuum")


plt.yscale('linear')


In [13]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan 1 ML tech. surface pre-injection, EGA in 80mm, dynamic vacuum")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(200, 2.5e-11, 'No emission', fontdict = font)
plt.text(525, 2.9e-11, '\u2193 Initial desorption peak', fontdict = font)
plt.text(1100, 2.6e-11, 'Grid transparent', fontdict = font)
plt.text(1670, 1.5e-11, 'Grid repelling', fontdict = font)
plt.text(2005, 4.46e-11, '\u2193 EGA rotated', fontdict = font)
plt.text(2306, 2.6e-11, 'Grid transparent', fontdict = font)
plt.yscale('log')

In [14]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA in 80mm, dynamic vacuum")


plt.yscale('log')


In [15]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution 1 ML tech. surface pre-injection, EGA in 80mm, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

In [16]:
## Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Elapsed time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution 1 ML tech. surface pre-injection, EGA in 80mm, dynamic vacuum")
plt.yscale('linear')

# Primary desorption calculation method
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.

Due to the EGA being fully retracted, it is safe to assume that the desorption takes place at approximately room temperature

In [17]:
#calculation delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = 293
print(T)
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)
#Emission current
#ignore the inrush current values
#hv["em"].where(hv["em"] <= 1.5e-6, np.nan, inplace=True)

I_em = np.nanmean(hv["em"].nlargest(30))
print(I_em)
#Conductance for CO in m3/s
c_co=10.13/1000
c_co2 = 8.08/1000


1.2560152475322217e-09
293
0.01992516581153798
8.3639e-07


In [18]:
#Desorption yield
eta_h2_tot = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The total desorption yield for H2 [mol/el]:",eta_h2_tot,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

The total desorption yield for H2 [mol/el]: 0.001184026165795611 



## Calculating desorption yield point-by-point

$$
\eta=\frac{C_{H2}\cdot (p_{n} - p_{n-1}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$


In [19]:
#partial pressure delta

p_delta = hv_aligned["hydrogen pp"].diff().fillna(hv_aligned["hydrogen pp"])*100
p_delta = p_delta.clip(lower=0)
print(p_delta)
#hv_shifted = hv_aligned.shift(periods=1)
#print(hv_shifted)
#p_delta = hv_aligned["hydrogen pp"] - hv_shifted["hydrogen pp"]
#print(hv_aligned["hydrogen pp"].describe())
#print(hv_shifted["hydrogen pp"].describe())
print(p_delta.describe())

datetime
2022-12-13 16:41:38    2.664372e-08
2022-12-13 16:41:45    0.000000e+00
2022-12-13 16:41:51    0.000000e+00
2022-12-13 16:41:57    3.717032e-11
2022-12-13 16:42:04    2.409997e-10
                           ...     
2022-12-13 17:29:42    1.104177e-10
2022-12-13 17:29:49    0.000000e+00
2022-12-13 17:29:55    7.956392e-11
2022-12-13 17:30:01    0.000000e+00
2022-12-13 17:30:08    3.279734e-12
Name: hydrogen pp, Length: 461, dtype: float64
count    4.610000e+02
mean     1.809630e-10
std      1.546310e-09
min      0.000000e+00
25%      0.000000e+00
50%      2.550904e-12
75%      1.181919e-10
max      2.664372e-08
Name: hydrogen pp, dtype: float64


In [20]:
#constants are already defined
#emission current and H2 partial pressures are now available at the same sampling rates
hv_aligned["em"].where(hv_aligned["em"] <= 2e-6, np.nan, inplace=True)
print(hv_aligned.describe())
I_n = hv_aligned["em"].replace(0, np.nan).dropna()

print(I_n)
#I_n.where(em <= 1.5e-6, np.nan, inplace=True)

eta = ((x_h2*p_delta*q_e)/(k_B*T*I_n)).dropna()
eta = eta.loc[eta.index[1]:]
print("desorption yields point-by-point:","\n", eta)

       Elapsed time       VMon1       VMon2       VMon3        VMon4  \
count    461.000000  461.000000  461.000000   461.00000   461.000000   
mean    1669.336124  168.007158   82.601518   823.95705   823.447852   
std      842.610361  235.571298   37.956151   379.90360   380.378359   
min      214.220000    0.040000    0.060000     0.12000     0.020000   
25%      942.410000    0.060000  100.060000   999.98000  1000.060000   
50%     1669.464000    0.120000  100.080000  1000.04000  1000.160000   
75%     2396.641000  499.920000  100.100000  1000.04000  1000.160000   
max     3118.554000  499.980000  100.140000  1000.12000  1000.220000   

             VMon5  VMon6  VMon7         VMon8       IMon1  ...       IMon4  \
count   461.000000  461.0  461.0  4.610000e+02  461.000000  ...  461.000000   
mean    734.420521    0.0    0.0  2.000000e-02   -0.035415  ...   26.384906   
std     440.737092    0.0    0.0  2.327055e-16    0.074513  ...   12.682600   
min       0.000000    0.0    0.0  2

In [21]:
#slicing eta to match hv_aligned["Elapsed time"]:
# Slice x using the index of y
hv_downsampled = hv_aligned.loc[eta.index, :]

In [22]:
#plotting point-by-point eta
plt.figure(figsize=(10,6))
plt.plot(hv_downsampled["Elapsed time"],eta,marker=".", markersize=4,label='eta')
#plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Electon stimulated desorption yield (M/el)')
plt.legend()
plt.title("ESD vs time, df.join(how=inner)")
ax = plt.gca()
ax.set_yscale('log')
ax.grid(which="major",axis="x")
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()